# Importing

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Knowledge Base Preparation

In [2]:
knowledge_base_df = pd.read_excel('FullChatbotDataset.xlsx')
fintech_df = knowledge_base_df[knowledge_base_df['Topic'] == 'Fintech']
artech_df = knowledge_base_df[knowledge_base_df['Topic'] == 'ArTech']
proptech_df = knowledge_base_df[knowledge_base_df['Topic'] == 'PropTech']
digitalization_df = knowledge_base_df[knowledge_base_df['Topic'] == 'Digitalization']

# Loading Classification Model

In [3]:
import pickle
label_map = {"Fintech": 0, "ArTech": 1, "PropTech": 2, "Digitalization": 3, "Irrelevant": 4}
classifier = pickle.load(open('classifier.pkl', 'rb'))
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))

In [4]:
def predict_topic(question):
    question_vector = vectorizer.transform([question])
    prediction = classifier.predict(question_vector)
    return prediction

def get_KB_df(class_id):
    if class_id == 0:
        return fintech_df
    elif class_id == 1:
        return artech_df
    elif class_id == 2:
        return proptech_df
    elif class_id == 3:
        return digitalization_df
    else:
        return None

# Chatbot Type 1: Question-Matching Chatbot

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def find_best_match(user_query, knowledge_base,confidence_threshold=0.2):
    """
    Match the user's query to the closest question in the knowledge base.
    Returns the best matching question and its corresponding answer.
    """
    # Extract the questions
    questions = knowledge_base["Question"].tolist()
    
    # Vectorize the questions and the user query
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(questions + [user_query])  # Add user query
    
    # Compute cosine similarity
    similarities = cosine_similarity(vectors[-1], vectors[:-1]).flatten()  # Compare query to questions
    best_match_idx = similarities.argmax()  # Find the index of the best match

    # Threshold for minimum similarity
    if similarities[best_match_idx] < confidence_threshold:
        return None, None
    
    # Get the corresponding question and answer
    best_question = questions[best_match_idx]
    best_answer = knowledge_base.iloc[best_match_idx]["Answer"]
    return best_question, best_answer

In [15]:
# Interactive chatbot
print("Fintech Matching QA Chatbot is ready! Type 'exit' to quit.")
while True:
    user_query = input("You: ")
    print("You: ", user_query)
    if user_query.lower() == "exit":
        print("Goodbye!")
        break

    # Predict the topic
    prediction = predict_topic(user_query)
    class_id = prediction[0]
    knowledge_base = get_KB_df(class_id)

    if knowledge_base is None:
        print("Bot: I'm sorry, the questions is out of my scope.")
        continue

    question, answer = find_best_match(user_query, knowledge_base)

    if answer is None:
        print("Bot: I'm sorry, I don't have an answer to that question.")
    else:
        print(f"Bot: {answer}")

Fintech Matching QA Chatbot is ready! Type 'exit' to quit.
You:  How does blockchain works?
Bot: Blockchain technology works by recording transactions in blocks, which are then linked in chronological order. Each block contains transaction data, a timestamp, and a cryptographic hash of the previous block.
You:  How blockchain facilitate security?
Bot: Blockchain ensures data security through cryptographic hashing, decentralization (reducing single points of failure), and consensus mechanisms that validate transactions before they are recorded.
You:  What is the weather like today?
Bot: I'm sorry, the questions is out of my scope.
You:  exit
Goodbye!


# Chatbot Type 2: using LLM

In [42]:
import zhipuai

# Initialize the ZhipuAI client with your API key
client = zhipuai.ZhipuAI(api_key="42a0d7e358685ad79f65fd0b65482ed0.F4pyNvJM8zdfC6rL")

def get_response(question,role="Intermediate", company_KB = pd.DataFrame()):
    # Construct the prompt
    prompt = question

    if company_KB.empty:
        if role == "Beginner":
            prompt = f"The user asked: '{prompt}'. Respond in a way that is simple and easy to understand for someone new to the topic. Use examples or analogies to make the concept clear. Avoid technical jargon or complexity."
        elif role == "Intermediate":
            prompt = f"The user asked: '{prompt}'. Provide a detailed but straightforward explanation with practical examples. Assume the user has some prior knowledge but avoid excessive technical jargon."
        elif role == "Advanced":
            prompt = f"The user asked: '{prompt}'. Provide a detailed and technical explanation suitable for an advanced audience. Include domain-specific terminology, in-depth analysis, and references to advanced concepts or research when relevant."
        elif role == "Corporate/Company":
            prompt = f"The user asked: '{prompt}'. Provide a response tailored for a business professional. Focus on practical applications, strategies, financial implications, and relevant industry trends or statistics. Keep the response concise and actionable."

    else:
        _, answer = find_best_match(question, company_KB)
        if answer is not None:
            prompt = f"The user asked: '{question}'. Your are the representative of Nexora Innovations. Based on the context provided in '{answer}', craft a clear and concise response that directly addresses the user's query while ensuring the tone remains professional and informative. Include relevant details and examples if necessary, and avoid straying from the provided context."
        else:
            return "I'm sorry, I don't have an answer to that question. Please contact our customer service for further assistance."


    # Make the API call
    response = client.chat.completions.create(
    model="glm-4",
    messages=[
        {
            "role": "assistant",
            "content": "I am chatglm-4"
        },
        {
            "role": "user",
            "content": prompt
        }
    ])

    # Extract and return the output
    output = response.choices[0].message.content
    return output

## Addon: Different Roles

In [30]:
get_response("How does blockchain works?", "Beginner")

"Imagine a large digital ledger that is duplicated many times across a network of computers. This ledger records all the transactions made with a certain type of digital currency or data. Blockchain is like a continuously updating spreadsheet that everyone in the network can see and contribute to, but once information is entered, it can't be changed or removed.\n\nHere's a simple analogy: Think of blockchain as a chain of blocks. Each block contains a list of transactions, and each block is linked to the one before it, like a chain. When a new transaction occurs, it gets put into a new block, and that block gets added to the chain.\n\nHere's how it works step by step:\n\n1. **Transaction**: Let's say you want to send some digital money to a friend. This transaction is broadcast to the network.\n\n2. **Verification**: Computers on the network, called nodes, check that your transaction is valid. They ensure you have the money you want to send and that the transaction isn't fraudulent.\n\

In [31]:
get_response("How does blockchain works?", "Intermediate")

"Blockchain is a decentralized technology that enables the recording and verification of transactions across multiple computers, commonly known as nodes, in a network. It works by creating a continuous chain of blocks, where each block contains a list of transactions. Here's a step-by-step explanation of how it works, along with practical examples:\n\n1. Transaction: A transaction is simply an exchange of data or assets between two parties. For example, if Alice wants to send Bob 1 Bitcoin, she would initiate a transaction.\n\n2. Block: Transactions are grouped into blocks. A block can contain hundreds or even thousands of transactions. Each block has a unique identifier called a hash and a timestamp.\n\n   - Example: Think of a block as a page in a ledger book where multiple transactions are recorded.\n\n3. Hashing: A hash function takes input data (like transactions in a block) and produces a fixed-size string of characters, which is the hash. This hash is unique to the input data, m

In [32]:
get_response("How does blockchain works?", "Advanced")

'Blockchain technology is a decentralized digital ledger that maintains a continuously growing list of records, called blocks, which are secured from tampering and revision. It is a foundational innovation underlying cryptocurrencies like Bitcoin, and its application extends beyond digital currencies to various domains, including supply chain management, smart contracts, and decentralized finance (DeFi).\n\nHere is a detailed, technical explanation of how blockchain works, tailored to an advanced audience:\n\n**1. Blockchain Structure:**\n\n   - **Block:** Each block contains a list of transactions, a timestamp, a nonce (a unique random number used to create a hash), and the hash of the previous block, creating a chain of blocks.\n   - **Hash Function:** A cryptographic hash function takes an input (or \'message\') and produces a fixed-size string, which is the hash. In a blockchain, each block\'s hash is computed based on the block\'s contents, including the previous block\'s hash.\n 

In [29]:
get_response("How does blockchain works?", "Corporate/Company")

'Blockchain works as a decentralized digital ledger that records transactions across multiple computers, creating an immutable and transparent record. Here\'s a concise explanation tailored for a business professional:\n\n"Blockchain operates by linking transactions in \'blocks\' through cryptographic algorithms, ensuring security and preventing tampering. Each block refers to the previous one, forming a \'chain.\' This technology\'s practical applications for business include:\n\n1. Supply Chain Management: Enhance transparency, track goods, and reduce fraud. For instance, Walmart has reduced the time to trace food items from days to seconds using blockchain.\n\n2. Smart Contracts: Self-executing contracts with predefined rules that automatically enforce obligations, saving time and legal fees in areas like real estate, logistics, and intellectual property.\n\n3. Financial Transactions: Offer secure, fast, and cost-effective cross-border payments. Ripple, for example, has partnered wi

# Addon 2: Company Specific Q&A

While the large language model does not know about specific company information, like new startups, we can use a knowledge base for company Q&A to assist the answer

In this section, we make up a fake company Nexora Innovations, and demonstrate how text processing can help LLM models.

In [40]:
company_df = pd.read_excel('CompanyInfo.xlsx')

In [43]:
get_response("What is your company?", company_KB= company_df)

'I am ChatGLM-4, an AI assistant, and I represent Nexora Innovations. Nexora is an AI-driven company that specializes in fintech, augmented reality technologies, and providing digital transformation solutions to help businesses evolve and thrive in the digital era. We leverage cutting-edge AI to develop innovative products and services that enhance efficiency and user experience in financial and AR applications.'

In [46]:
get_response("How to contact your company?", company_KB= company_df)

'You can contact Nexora Innovations by reaching out to us via email at support@nexora.com or by calling our hotline at +65-1234-5678. Our team is ready to assist you with any inquiries or support you may need.'

In [47]:
get_response("How does your company use AI?", company_KB= company_df)

'At Nexora Innovations, we leverage AI by developing custom solutions that are tailored to meet the unique needs of our clients across various industries. Our AI applications are designed to enhance business processes, improve decision-making, and optimize operations. For instance, we might use machine learning algorithms to help a manufacturing company predict equipment maintenance, or develop natural language processing tools for a customer service department to better understand and respond to client inquiries. By integrating AI into these workflows, Nexora enables businesses to achieve greater efficiency, accuracy, and cost savings.'